In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision import models
from sklearn.neighbors import KNeighborsClassifier

In [2]:
from src.lfw import LFW

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
lfw = LFW('../../storage/lfw', limit_to_A=False)
lfw_limit = LFW('../../storage/lfw', limit_to_A=True)

In [5]:
model = models.vgg16(pretrained=True)
model = model.to(device)
model.classifier[4] = nn.Sequential()
model.classifier[5] = nn.Sequential()
model.classifier[6] = nn.Sequential()
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
train_loader = torch.utils.data.DataLoader(lfw.train_data, batch_size=128, shuffle=False)
test_loader = torch.utils.data.DataLoader(lfw.test_data, batch_size=128, shuffle=False)

In [7]:
outputs = []
clazzes = []
model.eval()
with torch.no_grad():
    for index, (image, clazz, attributes) in enumerate(train_loader):
        image = image.to(device)
        output = model(image)
        outputs.append(output.cpu().numpy())
        clazzes.append(clazz)

In [8]:
o = np.concatenate(outputs)
c = np.concatenate(clazzes)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(o, c)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [34]:
correct = 0
num = 0
model.eval()
with torch.no_grad():
    for index, (image, clazz, attributes) in enumerate(test_loader):
        image = image.to(device)
        output = model(image)
        pred = knn.predict(output.cpu().numpy())
        pred = torch.tensor(pred)
        correct += torch.sum(np.equal(pred, clazz))
        num += clazz.shape[0]

print("%.4f%% ACC" % (correct.item()/num))

0.0857% ACC


In [32]:
correct.item()

7